#### 加载数据集

In [7]:
count = 0
alldata = []
with open("./all_docs.txt", "r", encoding="utf-8") as data:
    for line in data:
        line = line.strip()
        alldata.append(line)
        count += 1 
print(count)
print(len(alldata))

108295
108295


#### 获取训练集的id和对应的title_content

In [158]:
import jieba.analyse as aly
train_id_list = []
train_title_content_list = []
train_dict = {}  # key = id, value = title+"。"+content
for line in alldata:
    line = line.strip()
    if len(line.split("\001")) == 3:
        ids = line.split("\001")[0]
        title_content = line.split("\001")[1] + "。" + line.split("\001")[2]
        
        train_id_list.append(ids)
        train_title_content_list.append(title_content)
        
        train_dict[ids] = title_content

In [295]:
print(len(train_id_list))
print(len(train_title_content_list))
print(len(train_dict))

108295
108295
108295


### 获取测试集的id和对应的key words

In [156]:
'''
    获取测试集的数据id
'''
test_dict = {} # key = id, value = key words
test_id_list = [] # 只获取test data的id
test_keywords_list = []
with open("./train_docs_keywords.txt", "r", encoding="utf-8") as data:
    for line in data:
        line = line.strip()
        test_id_list.append(line.split("\t")[0])
        test_keywords_list.append(line.split("\t")[1])
        test_dict[line.split("\t")[0]] = line.split("\t")[1]

In [157]:
print(len(test_id_list))
print(len(test_keywords_list))
print(len(test_dict))

1000
1000
1000


In [130]:
test_id_list[0]

'D012650'

### 对测试集的文本数据做关键词提取

In [131]:
test_data = []
for i in test_id_list:
    for k, v in train_dict.items():
        if i == k:
            test_data.append(v)

In [160]:
len(test_data)

1000

In [133]:
import jieba
jieba.load_userdict("./user_dict.txt")
# jieba.add_word("三生三世", True)

##### tdidf提取

In [142]:
key_words_tfidf = []
for line in test_data:
    key_words_tfidf.append(aly.extract_tags(line)[:10])

In [143]:
key_words_tfidf[0]

['三生三世', '东华', '热巴', '桃花', '出演', '十里', '一部', '枕上', '剧情', '迪丽']

##### textrank提取

In [136]:
key_words_textrank = []
for line in test_data:
    key_words_textrank.append(aly.textrank(line))

In [137]:
key_words_textrank[0]

['是', '三生三世', '书', '剧情', '作品', '热巴', '桃花', '来', '知道', '制作']

#### 每篇测试集只提取tfidf和textrank结果重合部分的前2个

In [179]:
common_part = []
common = zip(key_words_tfidf, key_words_textrank)
for tfidf, textrank in common:
#     print(tfidf, textrank)
#     print("**************")
    con = [i for i in tfidf if i in textrank]
    common_part.append(','.join(con[:2]))   

In [180]:
len(common_part)

1000

In [181]:
common_part[0]

'三生三世,热巴'

In [152]:
# common_part

### 对比测试集的结果 vs tdidf/textrank取共同集合前2个获得的结果

In [194]:
# true keywords: test_keywords_list
# pred keywprds: common_part
score = 0
for true, pred in zip(test_keywords_list, common_part):
    score += len([keyword for keyword in true.split(",") if keyword in pred.split(",")]) * 0.5
print(score)

247.5


### 按照测试集每篇真实的关键词的个数，提取tfidf和textrank重合部分的结果

In [196]:
'''
    获取测试集中每篇文章实际的keywords的长度
'''
true_len_for_each_id = []
for i in test_keywords_list:
    true_len_for_each_id.append(len(i.split(",")))

In [198]:
len(true_len_for_each_id)

1000

In [230]:
'''
    获取tfidf和textrank重叠的结果
'''
common_part_2 = []
common = zip(key_words_tfidf, key_words_textrank)
for tfidf, textrank in common:
#     print(tfidf, textrank)
#     print("**************")
    con = [i for i in tfidf if i in textrank]
    common_part_2.append(','.join(con))   

In [222]:
'''
    按实际关键词的长度截取tfidf和textrank重叠的结果
'''
common_part_22 = []
for keywords, number in zip(common_part_2, true_len_for_each_id):
#     print(keywords.split(",")[:number])
    common_part_22.append(",".join(keywords.split(",")[:number]))

In [228]:
'''
    预测准确的关键词共620个
'''
true_keywords_num = 0
for true, pred in zip(test_keywords_list, common_part_22):
    true_keywords_num += len([keyword for keyword in true.split(",") if keyword in pred.split(",")]) 
print(true_keywords_num)

620


In [226]:
'''
    test_data中总共有2992个keywords
'''
from functools import reduce
reduce(lambda x,y : x+y, true_len_for_each_id)

2992

### 第一版答案，先按照tfidf和textrank结果，取重叠部分的前2个，作为最终的all docs的关键词

#### 加载数据集

In [231]:
count = 0
alldata = []
with open("./all_docs.txt", "r", encoding="utf-8") as data:
    for line in data:
        line = line.strip()
        alldata.append(line)
        count += 1 
print(count)
print(len(alldata))

108295
108295


#### 获取训练集的id和相应的title_content

In [232]:
import jieba.analyse as aly
train_id_list = []
train_title_content_list = []
train_dict = {}  # key = id, value = title+"。"+content
for line in alldata:
    line = line.strip()
    if len(line.split("\001")) == 3:
        ids = line.split("\001")[0]
        title_content = line.split("\001")[1] + "。" + line.split("\001")[2]
        
        train_id_list.append(ids)
        train_title_content_list.append(title_content)
        
        train_dict[ids] = title_content

In [233]:
print(len(id_list))
print(len(title_content_list))
print(len(train_dict))

108295
108295
108295


#### tfidf，textrank分别提取关键词，并取前2个重叠的关键词

In [254]:
import jieba.analyse as aly

# tfidf提取关键词
tfidf = []
for line in title_content_list:
    tfidf.append(aly.extract_tags(line))
print(len(tfidf))

108295


In [241]:
# textrank提取关键词
textrank = []
for num, line in enumerate(title_content_list):
    try:
        textrank.append(aly.textrank(line))
    except Exception as e:
        print(num+1)
        print(e)
print(len(textrank))

19146
float division by zero
21348
float division by zero
87502
float division by zero
108292


In [247]:
textrank.insert(19145,'')

In [251]:
textrank.insert(21347,'')

In [252]:
textrank.insert(87501,'')

In [253]:
len(textrank)

108295

In [ ]:
# textrank分词结果不太好，弃用。

### 加载自定义词典

### 直接取tfidf的结果前两个

In [276]:
tfidf_select_2 = []
for keywords in tfidf:
    tfidf_select_2.append(keywords[:2])

In [277]:
len(tfidf_select_2)

108295

In [304]:
from functools import reduce
tfidf_select_22 = []
for line in tfidf_select_2:
    tfidf_select_22.append(reduce(lambda x, y: x + "," + y, line)) 

In [305]:
tfidf_select_22[0]

'陈若仪,睫毛'

### 写成提交的格式

In [296]:
len(train_id_list)

108295

In [310]:
with open("./example.csv", "w", encoding="utf-8") as result:
    result.write("id" + "," + "lable1" + "," + "lable2" + "\n")
    for ids, keywords in zip(train_id_list, tfidf_select_22):
        result.write(ids + "," + keywords.strip() + "\n")